In [148]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.layers import Conv2D, BatchNormalization,Flatten, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input 
from tensorflow.keras.models import Model, load_model

In [2]:
"""Global parameters"""
H = 512
W = 512

In [5]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [7]:
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)

    return x, p

In [9]:
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [11]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    output = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, output, name="U_Net")

    return model

In [13]:
if __name__ == "__main__":
    input_shape = (512, 512, 3)
    model = build_unet(input_shape)
    model.summary()



Model: "U_Net"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 512, 512, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 512, 512, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 512, 512, 64)         0         ['batch_normalization[0]

In [15]:
smooth = 1e-15

In [17]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intesection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intesection
        x = (intesection + smooth) / (union + smooth)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [23]:
def dice_coef(y_true, y_pred):
    y_true = Flatten()(y_true)
    y_pred = Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    

In [25]:
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [113]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [29]:
np.random.seed(42)
tf.random.set_seed(42)

In [37]:
path = r"E:\python\segmentation\Computer Vision\Computer Vision\Data/"
create_dir(path + "Lung_Segmentation")

'directory has been created'

In [135]:
batch_size = 2
learning_rate = 1e-5
num_epochs = 50
model_path = os.path.join(path + "Lung_Segmentation/", "model.h5")
csv_path = os.path.join(path + "Lung_Segmentation/", "data.csv")

In [214]:
dataset_path = r"E:\python\segmentation\Computer Vision\Computer Vision\Data\NLM-MontgomeryCXRSet\MontgomerySet"

In [216]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "CXR_png", "*.png")))
    left_mask = sorted(glob(os.path.join(path, "ManualMask", "LeftMask", "*.png")))
    right_mask = sorted(glob(os.path.join(path, "ManualMask", "rightMask", "*.png")))

    split_size = int(len(images) * split)
    print("The len of images is : ", len(images))

    X_train, X_validation = train_test_split(images, test_size=split_size, random_state=42)
    X_train_left_mask, X_validation_left_mask = train_test_split(left_mask, test_size=split_size, random_state=42)
    X_train_right_mask, X_validation_right_mask = train_test_split(right_mask, test_size=split_size, random_state=42)
    
    X_train, X_test = train_test_split(X_train, test_size=split_size, random_state=42)
    X_train_left_mask, X_test_left_mask = train_test_split(X_train_left_mask, test_size=split_size, random_state=42)
    X_train_right_mask, X_test_right_mask = train_test_split(X_train_right_mask, test_size=split_size, random_state=42)

    return (X_train, X_train_left_mask, X_train_right_mask), (X_validation, X_validation_left_mask, X_validation_right_mask), (X_test, X_test_left_mask, X_test_right_mask)

In [218]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (H, W))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [220]:
def read_mask(path1, path2):
    x1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
    x2 = cv2.imread(path2, cv2.IMREAD_GRAYSCALE)
    x = x1 + x2
    x = cv2.resize(x, (H, W))
    x = x/np.max(x)
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [222]:
def tf_parse(x, y1, y2):
    def parse(x, y1, y2):
        x = x.decode()
        y1 = y1.decode()
        y2 = y2.decode()

        x = read_image(x)
        y = read_mask(y1, y2)
        return x, y

    x, y = tf.numpy_function(parse, [x, y1, y2], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [224]:
def tf_dataset(x, y1, y2, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y1, y2))
    dataset = dataset.shuffle(buffer_size=200)
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(4)
    return dataset

In [226]:
(X_train, X_train_left_mask, X_train_right_mask), (X_validation, X_validation_left_mask, X_validation_right_mask), (X_test, X_test_left_mask, X_test_right_mask) = load_data(dataset_path)

print(f"Train: {len(X_train)} - {len(X_train_left_mask)} - {len(X_train_right_mask)}")
print(f"Validation: {len(X_validation)} - {len(X_validation_left_mask)} - {len(X_validation_right_mask)}")
print(f"Test: {len(X_test)} - {len(X_test_left_mask)} - {len(X_test_right_mask)}")

The len of images is :  138
Train: 112 - 112 - 112
Validation: 13 - 13 - 13
Test: 13 - 13 - 13


In [121]:
x = read_image(X_train[0])
print(x.shape)

(512, 512, 3)


In [123]:
x = read_mask(X_train_left_mask[0], X_train_right_mask[0])
print(x.shape)

(512, 512, 1)


In [125]:
train_dataset = tf_dataset(X_train, X_train_left_mask, X_train_right_mask, batch=batch_size)
validation_dataset = tf_dataset(X_validation, X_validation_left_mask, X_validation_right_mask, batch=batch_size)

In [141]:
for x, y in train_dataset:
    print(x.shape[:5], y.shape[:5])

(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 3) (2, 512, 512, 1)
(2, 512, 512, 

Model

In [103]:
model = build_unet((H, W, 3))

In [127]:
metrics = [dice_coef, iou, Recall(), Precision()]

In [129]:
model.compile(loss=dice_loss, optimizer=Adam(learning_rate=learning_rate), metrics=metrics)

In [131]:
callbacks = [

    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path)
]

In [139]:
model.fit(
    train_dataset, 
    epochs=num_epochs,
    validation_data=validation_dataset,
    callbacks=callbacks
)

Epoch 1/50
56/56 [==============================] - ETA: 0s - loss: 0.5005 - dice_coef: 0.4995 - iou: 0.3357 - recall_4: 0.9476 - precision_4: 0.4108
Epoch 1: val_loss improved from inf to 0.63551, saving model to E:\python\segmentation\Computer Vision\Computer Vision\Data/Lung_Segmentation\model.h5


C:\Users\hamid\anaconda3\envs\MachineLearning\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


56/56 [==============================] - 562s 10s/step - loss: 0.5005 - dice_coef: 0.4995 - iou: 0.3357 - recall_4: 0.9476 - precision_4: 0.4108 - val_loss: 0.6355 - val_dice_coef: 0.3573 - val_iou: 0.2189 - val_recall_4: 1.0000 - val_precision_4: 0.2524 - lr: 1.0000e-05
Epoch 2/50
56/56 [==============================] - ETA: 0s - loss: 0.3655 - dice_coef: 0.6345 - iou: 0.4694 - recall_4: 0.9475 - precision_4: 0.5886
Epoch 2: val_loss improved from 0.63551 to 0.62960, saving model to E:\python\segmentation\Computer Vision\Computer Vision\Data/Lung_Segmentation\model.h5
56/56 [==============================] - 561s 10s/step - loss: 0.3655 - dice_coef: 0.6345 - iou: 0.4694 - recall_4: 0.9475 - precision_4: 0.5886 - val_loss: 0.6296 - val_dice_coef: 0.3715 - val_iou: 0.2292 - val_recall_4: 1.0000 - val_precision_4: 0.2539 - lr: 1.0000e-05
Epoch 3/50
56/56 [==============================] - ETA: 0s - loss: 0.2556 - dice_coef: 0.7444 - iou: 0.5971 - recall_4: 0.9709 - precision_4: 0.8176
E

In [152]:
model.save("Lung_Segmentation.h5")

In [252]:
results_path = r"E:\python\segmentation\Computer Vision\Computer Vision\Data\Lung_Segmentation/"
create_dir(results_path + "result")

In [260]:
my_model = r"E:\python\segmentation\Computer Vision\Computer Vision\Data\Lung_Segmentation\model.h5"

with CustomObjectScope({"iou":iou, "dice_coef":dice_coef, "dice_loss":dice_loss}):
    np.random.seed(42)
    tf.random.set_seed(42)
    model = tf.keras.models.load_model(my_model)

    #dataset_path = r""
    (X_train, _y_train1, y_train_2), (X_val, y_val_1, y_val_2), (X_test, y_test_1, y_test_2) = load_data(dataset_path) 

    for x, y1, y2 in tqdm(zip(X_test, y_test_1, y_test_2), total=len(X_test)):
        image_name = x.split("\\")[-1]
    
        print(image_name) # == MCUCXR_0017_0.png
    
        orig_x = cv2.imread(x, cv2.IMREAD_COLOR)
        orig_x = cv2.resize(orig_x, (W, H))
        x = orig_x / 255.0
        x = x.astype(np.float32)
        x = np.expand_dims(x, axis=0)
    
        orig_y1 = cv2.imread(y1, cv2.IMREAD_GRAYSCALE)
        orig_y2 = cv2.imread(y2, cv2.IMREAD_GRAYSCALE)
        orig_y = orig_y1 + orig_y2
        orig_y = cv2.resize(orig_y, (W, H))
        orig_y = np.expand_dims(orig_y, axis=-1)
        orig_y = np.concatenate([orig_y, orig_y, orig_y], axis=-1)

        y_pred = model.predict(x)[0] > 0.5
        y_pred = y_pred.astype(np.int32)

        save_path = r"E:\python\segmentation\Computer Vision\Computer Vision\Data\Lung_Segmentation\result"
        save_image = f"result{image_name}.png"
        #save_image_path = f"result{name+'A'}.png"
        save_image_path = os.path.join(save_path, save_image)
        #save_results(ori_x, ori_y, y_pred, save_image_path) 
        
        #s#ave_image_path = f"result{image_name}.png"
        
        y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
                                
    
        sep_line = np.ones((H, 10, 3)) * 255
        cat_image = np.concatenate([orig_x, sep_line, orig_y, sep_line, y_pred * 255], axis=1)
        cv2.imwrite(save_image_path, cat_image)
        

The len of images is :  138


  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

MCUCXR_0017_0.png
1/1 [==============================] - 4s 4s/step


  8%|██████▍                                                                            | 1/13 [00:04<00:53,  4.44s/it]

MCUCXR_0108_1.png
1/1 [==============================] - 1s 912ms/step


...

 85%|█████████████████████████████████████████████████████████████████████▍            | 11/13 [00:18<00:02,  1.42s/it]

MCUCXR_0097_0.png
1/1 [==============================] - 1s 938ms/step


 92%|███████████████████████████████████████████████████████████████████████████▋      | 12/13 [00:19<00:01,  1.43s/it]

MCUCXR_0015_0.png
1/1 [==============================] - 1s 935ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:21<00:00,  1.64s/it]
